# Creation of the CNN

### Importations 

In [5]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import pandas as pd
import time


from torch.autograd import Variable
from torch import nn, optim
from torch.nn import functional as F
from matplotlib import gridspec




print(torch.__version__)

1.0.1.post2


In [6]:
print("Tmux a vraiment run cette schisse.")

Tmux a vraiment run cette schisse.


### The Aim 

Some explation about the purpose of this project.

### The Data

Some insight about the data.

In [47]:
Dataset = pd.read_pickle("/home/janjar/Dataset/Trainingset/Training_dataframe.pkl")
Dataset.loc[0:10]

,Room,Array_position,Distance,Absorption,SNR,Audio_file,Phase_Matrix,Label
0,0,"[1.924007832111607, 3.3174552113304, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
1,0,"[3.395902870077362, 2.0557500170094034, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
2,0,"[1.526185199528145, 2.2726572315160185, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
3,0,"[2.6692549127850898, 3.4855723081023084, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
4,0,"[3.015681218831948, 1.643219467690823, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
5,0,"[1.556422618908361, 2.8311521189608246, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
6,0,"[1.616051796995994, 2.967584831240267, 1.5]",1.0,0.702471,0,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,1
7,0,"[1.5000025446100484, 2.4977440714501156, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
8,0,"[3.4733630593992753, 2.729269174981904, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0
9,0,"[1.7578897843542212, 1.829722126402657, 1.5]",1.0,0.702471,5,/home/janjar/Dataset/Trainingset/audio_signals...,/home/janjar/Dataset/Trainingset/phase_matrix/...,0


#### Defining our number of samples.

In [8]:
number_of_rooms = Dataset.shape[0]

### Loading the phase matrices in tensors with respect to the indexing shown in the dataframe.

In [9]:
#path containing the Training data on the machine you are working on.
#path = "/home/janjar/Dataset/Trainingset/"

data_matrix = np.zeros([number_of_rooms,4,129,390])
for i in range(number_of_rooms):
    fileName_matrix = Dataset.iloc[i]['Phase_Matrix']
    #print(fileName_matrix)
    fileObject2 = open(fileName_matrix, 'rb')
    matrix_loaded = pkl.load(fileObject2)
    fileObject2.close()
    data_matrix[i] = matrix_loaded
data_matrix.shape    

(140, 4, 129, 390)

In [10]:
data_matrix = torch.from_numpy(data_matrix)
data_matrix = data_matrix.view(-1,4,129,390)
data_matrix = data_matrix.type('torch.FloatTensor')
data_matrix.shape

torch.Size([140, 4, 129, 390])

In [13]:
final_data = torch.ones([140*390, 4,129], dtype=torch.float64)

In [14]:
i = 0
for setup in range(140):
     for nb_tf in range(390):
        final_data[i] = data_matrix[setup,:,:,nb_tf]
        i = i+1
    

In [15]:
final_data.shape

torch.Size([54600, 4, 129])

### Function that reshape a label into the right format for the data_targets.

In [7]:
def prep_targets(index,label):
    target = torch.tensor(label)
    target = target.expand(390,1)
    return target

In [17]:
data_targets = torch.zeros([number_of_rooms], dtype=torch.float64)
for i in range(number_of_rooms):
    data_targets[i] = Dataset.iloc[i]['Label']
data_targets.shape   

torch.Size([140])

In [18]:
data_targets[0]

tensor(1., dtype=torch.float64)

In [57]:
final_targets = torch.ones([140*390], dtype=torch.float64)
test = []
for i in range(140):
    curr = [data_targets[i].item()]*390
    test.append(curr)
    

In [60]:
final_targets = torch.FloatTensor(test)
final_targets = final_targets.flatten()
final_targets.shape

torch.Size([54600])

# Architecture 1.0 : vanilla CNN for DOA 

 In this section, we will focus on building a vanilla CNN in order to recognize the directions of arrival of the   sound in a specific room. Once done, we will then compare "by hand" the labelel dataset to the predicted values. This is the most basic setup and will try to improve latter.

## 1.1 Creating Model

In [33]:
class Net(nn.Module):
    def __init__(self,nb_hidden = 50, n = 4):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=2)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=2)
        self.conv3 = nn.Conv2d(8 ,8, kernel_size=2)
        self.fc1 = nn.Linear(390096, nb_hidden)# (1x8064) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(nb_hidden*4,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,2)
        
    
    def forward(self, x, n):
        #print("What actually enters the model:",x.shape)
        test =  x[:,:,0,:,:].view(-1,1,129,390)
        #print("Shape of test:",test.shape)
        output = torch.zeros([390096,1])
        for i in range(n):
            
            y = x[:,:,i,:,:].view(-1,1,129,390)
            y = F.relu(self.conv1(y)) 
            y = F.relu(self.conv2(y))
            y = F.relu(self.conv3(y))
            y = F.relu(self.fc1(y.view(-1, 390096)))
            
            if (i==0):
                output = y
            else:
                output = torch.cat((output,y),1)
                
        output = F.relu(self.fc2(output))
        output = self.dropout(output)
        output = F.softmax(self.fc3(output))        

        return output            

In [119]:
class ws_Net(nn.Module):
    def __init__(self,nb_hidden = 50, n = 390 ):
        super(ws_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=2)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=2)
        self.conv3 = nn.Conv2d(8 ,8, kernel_size=2)
        self.fc1 = nn.Linear(1008, nb_hidden)# (1x1008) being the dim of the censor obtained by flattening the output of the 3rd CL.
        self.fc2 = nn.Linear(nb_hidden*n,512)
        self.fc3 = nn.Linear(512,2)
        
    
    def forward(self, x, n):
        
        #print("What actually enters the model:",x.shape)
        test =  x[:,:,:,:,0].view(-1,1,4,129)
        #print("Shape of test:",test.shape)
        output = torch.zeros([1008,1])
        
        for i in range(n):
            y = x[:,:,:,:,i].view(-1,1,4,129)
            y = F.relu(self.conv1(y)) 
            y = F.relu(self.conv2(y))
            y = F.relu(self.conv3(y))
            y = F.relu(self.fc1(y.view(-1, 1008)))
            
            if (i==0):
                output = y
            else:
                output = torch.cat((output,y),1)
                
        #print("Shape of the out of fc1:",output.shape)        
                
        output = F.relu(self.fc2(output))
        
        #print("Shape of the out of fc2:",output.shape) 

        output = F.softmax(self.fc3(output))   
        
        return output  

In [93]:
class SimpleModel(nn.Module):

    def __init__(self, nb_hidden=100):
        super(SimpleModel, self).__init__()
        self.cl1 = nn.Conv3d(1, 8, kernel_size=(2,2,2),stride=1)
        self.cl2 = nn.Conv3d(8, 16, kernel_size=(2,2,2),stride=1)
        self.cl3 = nn.Conv3d(16, 32, kernel_size=(2,2,2),stride=1)
        self.fc1 = nn.Linear(4032, 200)
        self.fc2 = nn.Linear(200,nb_hidden)
        self.fc3 = nn.Linear(nb_hidden,37)
        #self.fc4 = nn.Linear(256,37)
 
 
    def forward(self, x):
       
        x = F.relu(self.cl1(x))
        x = F.relu(self.cl2(x))
        x = F.relu(self.cl3(x))
        print("Output of convlayer",x.shape)
        x = F.relu(self.fc1(x.view(-1, 4032)))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        print("Output of the model shape:",x.shape)

        return x

    def compute_nb_errors(self,model, data_input, data_target, mini_batch_size):

        nb_data_errors = 0
        for b in range(0, data_input.size(0), mini_batch_size):
            a = model(data_input.narrow(0, b, mini_batch_size))
            val = torch.max(a,1)[1]
            for k in range(mini_batch_size):
                if data_target.data[b + k] != val[k]:
                    nb_data_errors = nb_data_errors + 1

        return nb_data_errors

### Preparing the Data

In [94]:
def prep_input_vanilla(train_input):
    new_train_input = train_input.view(-1,1,4,129,390)
    return new_train_input

In [95]:
def prep_input_model2(train_input):
    new_train_input = train_input.view(-1,1,4,129,390)
    return new_train_input

In [96]:
train_data = prep_input_vanilla(data_matrix)
train_data.shape

torch.Size([140, 1, 4, 129, 390])

In [97]:
train_data = prep_input_model2(data_matrix)
train_data.shape

torch.Size([140, 1, 4, 129, 390])

In [98]:
def prep_labels_vanilla(train_input):
    new_train_input = train_input.view(-1)
    return new_train_input

In [99]:
train_targets = prep_labels_vanilla(data_targets)
train_targets = train_targets.to(dtype=torch.int64)
train_data, train_targets = Variable(train_data), Variable(train_targets)
train_data[:,:,0,:,:].shape

torch.Size([140, 1, 129, 390])

### Spliting the Training Data into Training/Validation.

In [100]:
training_data = train_data[0:120]
training_targets = train_targets[0:120]
validation_data = train_data[110:140]
validation_targets = train_targets[110:140]
training_data.shape, validation_data.shape, training_targets.shape, validation_targets.shape,len(training_targets),len(validation_targets)

(torch.Size([120, 1, 4, 129, 390]),
 torch.Size([30, 1, 4, 129, 390]),
 torch.Size([120]),
 torch.Size([30]),
 120,
 30)

In [101]:
train_targets

tensor([1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0])

### End of Sanity check.

## 1.2 Training the model

In [120]:
model = Net()

In [121]:
model1 = ws_Net()

In [122]:
model2 = SimpleModel()

In [123]:
mini_batch_size = 10
nb_epochs = 10
eta = 0.0001 #learning rate
#criterion = nn.MSELoss() # MeanSquaredloss
criterion = torch.nn.CrossEntropyLoss() #Cross Entropy
optimizer = torch.optim.SGD(model.parameters(), lr = eta, momentum = 0.95) #Stochastic gradient descent
optimizer1 = torch.optim.Adam(model.parameters(), lr = eta, weight_decay=0.01) #ADAM
net_ch = 4
ws_ch = 390

### Need to figure out how to use mini-batches to gain time in this configuration.
#### train_model is the method used to update the weights.

In [124]:
def train_model_3(model, optimizer, nb_epochs, train_input, train_target ,mini_batch_size,n):

    start = time.time()
    for e in range(0,nb_epochs):
        start_ep = time.time()
        for b in range(0, train_input.size(0), mini_batch_size):
            start = time.time()
            #print("Shape of the input of the model:",train_input.narrow(0, b, mini_batch_size).shape)
            output = model(train_input.narrow(0, b, mini_batch_size),n)
            target = train_target.narrow(0, b, mini_batch_size)
            indices = output.argmax(1)
            print("batch :",b," Output :",output,"Targets:", target)
            loss = criterion(output,target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if (e == 0):
            end_ep = time.time()
            print("Time for 1 epochs is :{:5}".format(-(start_ep-end_ep)))
            
        print("Loss for epoch{:3} is {:5} ".format(e,loss.data.item()))
            
    end = time.time()
    print("Time the hole training is :{:5}".format(-(start-end)))

       

    

In [115]:
train_model_3(model,optimizer1,nb_epochs,training_data,training_targets,mini_batch_size,net_ch)

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


batch : 0  Output : tensor([[0.4915, 0.5085],
        [0.4889, 0.5111],
        [0.4861, 0.5139],
        [0.4847, 0.5153],
        [0.4869, 0.5131],
        [0.4908, 0.5092],
        [0.4909, 0.5091],
        [0.4889, 0.5111],
        [0.4885, 0.5115],
        [0.4887, 0.5113]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 1, 1, 1, 0, 1, 1, 0, 0, 0])
batch : 10  Output : tensor([[0.4030, 0.5970],
        [0.4243, 0.5757],
        [0.3949, 0.6051],
        [0.3886, 0.6114],
        [0.3996, 0.6004],
        [0.3891, 0.6109],
        [0.3989, 0.6011],
        [0.3839, 0.6161],
        [0.3892, 0.6108],
        [0.4131, 0.5869]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])
batch : 20  Output : tensor([[0.3229, 0.6771],
        [0.3355, 0.6645],
        [0.3397, 0.6603],
        [0.3522, 0.6478],
        [0.3459, 0.6541],
        [0.3368, 0.6632],
        [0.3318, 0.6682],
        [0.3539, 0.6461],
        [0.3233, 0.6767],
        [0.3322, 0.6678]], grad_

batch : 110  Output : tensor([[0.6738, 0.3262],
        [0.6302, 0.3698],
        [0.5922, 0.4078],
        [0.6145, 0.3855],
        [0.6456, 0.3544],
        [0.6340, 0.3660],
        [0.6562, 0.3438],
        [0.5602, 0.4398],
        [0.5623, 0.4377],
        [0.5691, 0.4309]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 0])
Loss for epoch  1 is 0.6667132377624512 
batch : 0  Output : tensor([[0.3960, 0.6040],
        [0.6199, 0.3801],
        [0.5498, 0.4502],
        [0.4959, 0.5041],
        [0.7630, 0.2370],
        [0.4959, 0.5041],
        [0.4568, 0.5432],
        [0.6933, 0.3067],
        [0.7178, 0.2822],
        [0.7026, 0.2974]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 1, 1, 1, 0, 1, 1, 0, 0, 0])
batch : 10  Output : tensor([[0.6461, 0.3539],
        [0.6878, 0.3122],
        [0.6774, 0.3226],
        [0.7459, 0.2541],
        [0.6993, 0.3007],
        [0.5431, 0.4569],
        [0.6265, 0.3735],
        [0.7410, 0.2590],
        [0.5970,

batch : 100  Output : tensor([[0.6960, 0.3040],
        [0.2542, 0.7458],
        [0.8404, 0.1596],
        [0.4436, 0.5564],
        [0.5588, 0.4412],
        [0.8164, 0.1836],
        [0.3008, 0.6992],
        [0.8185, 0.1815],
        [0.7461, 0.2539],
        [0.8640, 0.1360]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 1, 0, 1, 1, 0, 1, 0, 0, 0])
batch : 110  Output : tensor([[0.7525, 0.2475],
        [0.7317, 0.2683],
        [0.2205, 0.7795],
        [0.6252, 0.3748],
        [0.8513, 0.1487],
        [0.3457, 0.6543],
        [0.8424, 0.1576],
        [0.2871, 0.7129],
        [0.6739, 0.3261],
        [0.8983, 0.1017]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 0])
Loss for epoch  3 is 0.5009082555770874 
batch : 0  Output : tensor([[0.6643, 0.3357],
        [0.1930, 0.8070],
        [0.3663, 0.6337],
        [0.2908, 0.7092],
        [0.7087, 0.2913],
        [0.2660, 0.7340],
        [0.3405, 0.6595],
        [0.8335, 0.1665],
        [0.8651

batch : 90  Output : tensor([[0.0565, 0.9435],
        [0.6931, 0.3069],
        [0.0865, 0.9135],
        [0.8132, 0.1868],
        [0.1264, 0.8736],
        [0.8968, 0.1032],
        [0.8653, 0.1347],
        [0.9359, 0.0641],
        [0.8926, 0.1074],
        [0.0784, 0.9216]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 0, 1, 0, 1, 0, 0, 0, 0, 1])
batch : 100  Output : tensor([[0.9288, 0.0712],
        [0.0650, 0.9350],
        [0.7941, 0.2059],
        [0.1338, 0.8662],
        [0.1672, 0.8328],
        [0.9172, 0.0828],
        [0.0707, 0.9293],
        [0.9323, 0.0677],
        [0.8387, 0.1613],
        [0.9363, 0.0637]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 1, 0, 1, 1, 0, 1, 0, 0, 0])
batch : 110  Output : tensor([[0.9438, 0.0562],
        [0.9707, 0.0293],
        [0.2530, 0.7470],
        [0.0617, 0.9383],
        [0.9499, 0.0501],
        [0.1388, 0.8612],
        [0.8780, 0.1220],
        [0.1047, 0.8953],
        [0.9668, 0.0332],
        [0.9338, 0.0662]], gr

batch : 80  Output : tensor([[0.0286, 0.9714],
        [0.9824, 0.0176],
        [0.0856, 0.9144],
        [0.0742, 0.9258],
        [0.9849, 0.0151],
        [0.9859, 0.0141],
        [0.9612, 0.0388],
        [0.0655, 0.9345],
        [0.9851, 0.0149],
        [0.1029, 0.8971]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 0, 1, 1, 0, 0, 0, 1, 0, 1])
batch : 90  Output : tensor([[0.0596, 0.9404],
        [0.9798, 0.0202],
        [0.1152, 0.8848],
        [0.9193, 0.0807],
        [0.1806, 0.8194],
        [0.9837, 0.0163],
        [0.9820, 0.0180],
        [0.9781, 0.0219],
        [0.9826, 0.0174],
        [0.0742, 0.9258]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 0, 1, 0, 1, 0, 0, 0, 0, 1])
batch : 100  Output : tensor([[0.9218, 0.0782],
        [0.0383, 0.9617],
        [0.9439, 0.0561],
        [0.0519, 0.9481],
        [0.0468, 0.9532],
        [0.8403, 0.1597],
        [0.0327, 0.9673],
        [0.9528, 0.0472],
        [0.9658, 0.0342],
        [0.9207, 0.0793]], gra

batch : 70  Output : tensor([[0.0141, 0.9859],
        [0.9846, 0.0154],
        [0.9771, 0.0229],
        [0.9630, 0.0370],
        [0.9836, 0.0164],
        [0.0291, 0.9709],
        [0.9828, 0.0172],
        [0.9512, 0.0488],
        [0.9651, 0.0349],
        [0.9067, 0.0933]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0])
batch : 80  Output : tensor([[0.0696, 0.9304],
        [0.9873, 0.0127],
        [0.0225, 0.9775],
        [0.0159, 0.9841],
        [0.9633, 0.0367],
        [0.9846, 0.0154],
        [0.8752, 0.1248],
        [0.0148, 0.9852],
        [0.9846, 0.0154],
        [0.0174, 0.9826]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 0, 1, 1, 0, 0, 0, 1, 0, 1])
batch : 90  Output : tensor([[0.0381, 0.9619],
        [0.9875, 0.0125],
        [0.0189, 0.9811],
        [0.9882, 0.0118],
        [0.0371, 0.9629],
        [0.9877, 0.0123],
        [0.9866, 0.0134],
        [0.9938, 0.0062],
        [0.9894, 0.0106],
        [0.1252, 0.8748]], grad

In [126]:
t = model(validation_data[10:20],4)
t

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.5331, 0.4669],
        [0.5286, 0.4714],
        [0.5245, 0.4755],
        [0.5264, 0.4736],
        [0.5281, 0.4719],
        [0.5284, 0.4716],
        [0.5246, 0.4754],
        [0.5336, 0.4664],
        [0.5213, 0.4787],
        [0.5343, 0.4657]], grad_fn=<SoftmaxBackward>)

In [71]:
validation_targets[10:20]

tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 0])

In [125]:
train_model_3(model1,optimizer1,nb_epochs,training_data,training_targets,mini_batch_size,ws_ch)

/home/janjar/test/lib/python3.5/site-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


batch : 0  Output : tensor([[0.5145, 0.4855],
        [0.5141, 0.4859],
        [0.5137, 0.4863],
        [0.5115, 0.4885],
        [0.5135, 0.4865],
        [0.5137, 0.4863],
        [0.5147, 0.4853],
        [0.5143, 0.4857],
        [0.5114, 0.4886],
        [0.5147, 0.4853]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 1, 1, 1, 0, 1, 1, 0, 0, 0])
batch : 10  Output : tensor([[0.5154, 0.4846],
        [0.5128, 0.4872],
        [0.5108, 0.4892],
        [0.5109, 0.4891],
        [0.5123, 0.4877],
        [0.5158, 0.4842],
        [0.5153, 0.4847],
        [0.5151, 0.4849],
        [0.5146, 0.4854],
        [0.5148, 0.4852]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])
batch : 20  Output : tensor([[0.5152, 0.4848],
        [0.5166, 0.4834],
        [0.5146, 0.4854],
        [0.5153, 0.4847],
        [0.5168, 0.4832],
        [0.5172, 0.4828],
        [0.5140, 0.4860],
        [0.5166, 0.4834],
        [0.5173, 0.4827],
        [0.5128, 0.4872]], grad_

batch : 110  Output : tensor([[0.5157, 0.4843],
        [0.5119, 0.4881],
        [0.5134, 0.4866],
        [0.5175, 0.4825],
        [0.5140, 0.4860],
        [0.5127, 0.4873],
        [0.5130, 0.4870],
        [0.5163, 0.4837],
        [0.5098, 0.4902],
        [0.5159, 0.4841]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 0])
Loss for epoch  1 is 0.6912064552307129 
batch : 0  Output : tensor([[0.5145, 0.4855],
        [0.5141, 0.4859],
        [0.5137, 0.4863],
        [0.5115, 0.4885],
        [0.5135, 0.4865],
        [0.5137, 0.4863],
        [0.5147, 0.4853],
        [0.5143, 0.4857],
        [0.5114, 0.4886],
        [0.5147, 0.4853]], grad_fn=<SoftmaxBackward>) Targets: tensor([1, 1, 1, 1, 0, 1, 1, 0, 0, 0])
batch : 10  Output : tensor([[0.5154, 0.4846],
        [0.5128, 0.4872],
        [0.5108, 0.4892],
        [0.5109, 0.4891],
        [0.5123, 0.4877],
        [0.5158, 0.4842],
        [0.5153, 0.4847],
        [0.5151, 0.4849],
        [0.5146,

batch : 100  Output : tensor([[0.5158, 0.4842],
        [0.5141, 0.4859],
        [0.5137, 0.4863],
        [0.5162, 0.4838],
        [0.5144, 0.4856],
        [0.5086, 0.4914],
        [0.5165, 0.4835],
        [0.5120, 0.4880],
        [0.5161, 0.4839],
        [0.5141, 0.4859]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 1, 0, 1, 1, 0, 1, 0, 0, 0])
batch : 110  Output : tensor([[0.5157, 0.4843],
        [0.5119, 0.4881],
        [0.5134, 0.4866],
        [0.5175, 0.4825],
        [0.5140, 0.4860],
        [0.5127, 0.4873],
        [0.5130, 0.4870],
        [0.5163, 0.4837],
        [0.5098, 0.4902],
        [0.5159, 0.4841]], grad_fn=<SoftmaxBackward>) Targets: tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 0])
Loss for epoch  3 is 0.6912064552307129 
batch : 0  Output : tensor([[0.5145, 0.4855],
        [0.5141, 0.4859],
        [0.5137, 0.4863],
        [0.5115, 0.4885],
        [0.5135, 0.4865],
        [0.5137, 0.4863],
        [0.5147, 0.4853],
        [0.5143, 0.4857],
        [0.5114

KeyboardInterrupt: 